# Linear Regression

In [ ]:
import pandas as pd
import numpy as np
import sklearn
#from sklearn import linear_model
import matplotlib.pyplot as pyplot
import pickle
from matplotlib import style

data = pd.read_csv("student-mat.csv", sep=";")
print(data.head())

In [ ]:
data = data[["G1", "G2", "G3", "studytime", "failures", "absences"]]
print(data.head())

In [ ]:
predict = "G3"

X = np.array(data.drop([predict], axis=1))
y = np.array(data[predict])

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)


"""best = 0
for _ in range(30):

    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)

    linear = linear_model.LinearRegression()

    linear.fit(x_train, y_train)

    acc = linear.score(x_test, y_test)
    print(acc)

    if acc > best:
        best = acc
        # saves model
        with open("studentmodel.pickle", "wb") as f:
            pickle.dump(linear, f)"""

In [ ]:
pickle_in = open("studentmodel.pickle", "rb")
linear = pickle.load(pickle_in)

In [ ]:
print("Coefficient:", linear.coef_)
print("Intercept:", linear.intercept_)


In [ ]:
predictions = linear.predict(x_test)

for x in range(len(predictions)):
    print(predictions[x], x_test[x], y_test[x])

In [ ]:
p = "absences"
style.use("ggplot")
pyplot.scatter(data[p], data["G3"])
pyplot.xlabel(p)
pyplot.ylabel("Final Grade")
pyplot.show()

# Classification - K Nearest Neighbors (KNN)

In [ ]:
import sklearn
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing

data = pd.read_csv("car.data")
print(data.head())

In [ ]:
le = preprocessing.LabelEncoder()
buying = le.fit_transform(list(data["buying"]))
maint = le.fit_transform(list(data["maint"]))
door = le.fit_transform(list(data["door"]))
persons = le.fit_transform(list(data["persons"]))
lug_boot = le.fit_transform(list(data["lug_boot"]))
safety = le.fit_transform(list(data["safety"]))
clss = le.fit_transform(list(data["class"]))
print(buying)

In [ ]:
predict = "class"

X = list(zip(buying, maint, door, persons, lug_boot, safety))
y = list(clss)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)

In [ ]:
model = KNeighborsClassifier(n_neighbors=9)

model.fit(x_train, y_train)
acc = model.score(x_test, y_test)
print(acc)

In [27]:
predicted = model.predict(x_test)
names = ["unacc", "acc", "good", "vgood"]

for x in range(len(predicted)):
    print("Predicted: ", names[predicted[x]], "Data: ", tuple(int(val) for val in x_test[x]), "Actual: ", names[y_test[x]])
    n = model.kneighbors([x_test[x]], 9, True)
    print("N: ", n)

Predicted:  unacc Data:  (3, 2, 2, 1, 2, 0) Actual:  unacc
N:  (array([[1., 1., 1., 1., 1., 1., 1., 1., 1.]]), array([[ 420,  932,  808, 1518,  614, 1335,  838,  728, 1453]]))
Predicted:  good Data:  (3, 0, 0, 1, 1, 2) Actual:  good
N:  (array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.41421356]]), array([[ 366,  894,  984, 1220,  609,  737,  383,  763,  910]]))
Predicted:  unacc Data:  (0, 1, 2, 1, 0, 2) Actual:  unacc
N:  (array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.41421356]]), array([[1473,  683,  388, 1530, 1489,  902,  316, 1416, 1520]]))
Predicted:  good Data:  (0, 3, 2, 2, 2, 2) Actual:  good
N:  (array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.41421356, 1.41421356]]), array([[ 706, 1065,  839,  996,  715,  545,  148,  658,  298]]))
Predicted:  good Data:  (0, 1, 2, 1, 0, 1) Actual:  good
N: 